# ARD Overpass Predictor <img align="right" src="../Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with the `DEA Sandbox` environment
* **Special requirements:** This notebook loads data from an external .csv file (`overpass_input.csv`) from the `Supplementary_data` folder of this repository


## Background
Knowing the time of a satellite overpass (OP) at a field site is necessary to plan field work activities. While predicting the timesteps for a single field site that receives only one overpass from a given satellite is easy, it gets more complicated when you need overpass information across multiple field sites, with sites that may lie in overlapping satellite passes. 
This notebook can be used to output a list of ordered overpasses for given field sites by providing an initial timestamp for field sites of interest. Output for multiple sites are ordered by date such that dual overpasses can be identified.

Caution: manual process! You need to provide a start date + time for the overpass of the site you are interested in - go to https://nationalmap.gov.au/ and https://evdc.esa.int/orbit/ to get an overpass for your location (lat/long) and add this to the input file.

See "overpass_input.xlsx" as an example input file - this contains a number of field sites, some with multiple overpasses in a given orbital period.

## Description
Make changes to the notebook, following the **Steps** in bold
1. Provide an input file - this can be the example included or your own field site information
2. Specify which field sites receive extra overpasses in an orbital period, ie those which lie in satellite imaging overlap
3. Specify output field sites - you can select a subset of sites of interest (if adding your site to the original example file)
4. Combine field site overpasses, need to enter these manually according to dataframe length, ie by field sites that receive most overpasses to least
5. Specify an output file name

### Load packages


In [1]:
# Import necessary tools

import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import os

### Step 1 - specify input file containing initial overpass data for a given site:

Read in base overpass file ```overpass_input.csv``` and specify datetime formats if necessary (if you get errors or times look odd, date formatting is needed)

- Input date times must be in UTC
- Make sure the necessary dates for your site are imported correctly - should be YYYY-MM-DD HH:MM:SS in 24hr format
- If you have weird .xlsx date formatting, pd.to_datetime can force formatting - see below

In [2]:
# Read in base file, must specify datetimes for those columns which aren't already in datetime format.
# An example file, 'overpass_input.csv' is provided. 
# Edit this file with new field sites, or load your own file with field sites as needed.

os.chdir('/home/jovyan/Supplementary_data/ARD_overpass_predictor/')  # change directory to location of input example file. 
overpass = pd.read_csv('overpass_input.csv', index_col='Site') # read in example file 'overpass_input.csv'

# Format date time for each field site. Secondary overpasses also need to be specified. Ie 1st L8 OP = "landsat_8", 
# second L8 OP = "landsat_8_2" and so on
overpass['landsat_8'] = pd.to_datetime(overpass.landsat_8)
overpass['sentinel_2a'] = pd.to_datetime(overpass.sentinel_2a)
overpass['sentinel_2b'] = pd.to_datetime(overpass.sentinel_2b)
overpass['landsat_8_2'] = pd.to_datetime(overpass.landsat_8_2, dayfirst=True, format='%d/%m/%Y %H:%M')
overpass['sentinel_2a_2'] = pd.to_datetime(overpass.sentinel_2a_2, dayfirst=True, format='%d/%m/%Y %H:%M')
overpass['sentinel_2b_2'] = pd.to_datetime(overpass.sentinel_2b_2, dayfirst=True, format='%d/%m/%Y %H:%M')

overpass
# overpass = base file - check if sensible. "NaT" values are expected - indicates a date was not entered in input file. 

,Latitude,Longitude,Path,Row,landsat_8,landsat_8_2,sentinel_2a,sentinel_2a_2,sentinel_2b,sentinel_2b_2
Site,,,,,,,,,,
Blanchetown,-34.206,139.63000,97,84,2019-01-12 00:27:00,NaT,2019-10-12 00:42:00,NaT,2019-08-12 00:39:00,2019-12-11 00:49:00
Dharawal,-34.151,150.55320,89,84,2019-12-15 23:43:00,2019-12-08 23:44:00,2019-08-12 00:01:00,NaT,2019-05-12 23:59:00,NaT
Dookie,-36.385,145.71899,93,85,2019-05-12 00:02:00,2019-12-13 00:03:00,2019-11-12 00:12:00,2019-12-04 00:22:00,2019-02-12 00:20:00,NaT
Fowlers_Gap,-31.066,141.75100,96,81,2019-10-12 00:19:00,2019-12-17 00:25:00,2019-10-12 00:41:00,2019-12-17 00:32:00,2019-12-18 00:38:00,NaT
Lake_George,-35.094,149.46300,90,84,2019-11-29 23:43:00,NaT,2019-12-20 23:58:00,NaT,2019-12-15 23:58:00,NaT
Lake_Lefroy,-31.342,121.64700,109,82,2019-05-12 01:40:00,NaT,2019-11-12 01:52:00,2019-12-14 02:02:00,2019-12-22 01:58:00,NaT
Mitchell_Downs,-23.517,144.31700,96,76,2019-10-12 00:17:00,NaT,2019-12-17 00:30:00,NaT,2019-12-18 00:36:00,NaT
Mullion,-35.123,148.86200,91,84,2019-11-29 23:43:00,2019-11-20 23:49:00,2019-12-20 23:58:00,2019-12-14 00:07:00,2019-12-15 23:58:00,2019-12-19 00:07:00
Narrabundah,-35.334,149.14500,90,85,2019-11-29 23:43:00,2019-11-20 23:49:00,2019-12-20 23:58:00,NaT,2019-12-15 23:58:00,NaT


In [3]:
# Set satellite timesteps for overpasses, calculated from https://evdc.esa.int/orbit/
# Depending on your application, you may want to add more accurate timesteps - these are hashed out as rough times are ok

l8_timestep = datetime.timedelta (days=16) #(days=15,hours=23,minutes=59,seconds=37)
s2a_timestep = datetime.timedelta (days=10) #(days=10,hours=0,minutes=1,seconds=10)
s2b_timestep = datetime.timedelta (days=10) #(days=10,hours=0,minutes=1,seconds=47)

In [4]:
# Specify start date

l8_startdate = overpass['landsat_8'] 
l8_startdate_2 = overpass['landsat_8_2'] 
sentinel2a_startdate = overpass['sentinel_2a']
sentinel2a_2_startdate = overpass['sentinel_2a_2']
sentinel2b_startdate = overpass['sentinel_2b']
sentinel2b_2_startdate = overpass['sentinel_2b_2']
sentinel2a_2_startdate

Site
Blanchetown                      NaT
Dharawal                         NaT
Dookie           2019-12-04 00:22:00
Fowlers_Gap      2019-12-17 00:32:00
Lake_George                      NaT
Lake_Lefroy      2019-12-14 02:02:00
Mitchell_Downs                   NaT
Mullion          2019-12-14 00:07:00
Narrabundah                      NaT
Pinnacles                        NaT
Tumbarumba       2019-12-11 00:12:00
Winton                           NaT
Name: sentinel_2a_2, dtype: datetime64[ns]

In [5]:
# Landsat 8 overpass prediction for 20 * the overpass frequency - ie 20*l8_timestep = 320 days

landsat = list()
for i in range(20):
    landsat.append(l8_startdate + l8_timestep*(i))

landsat = pd.DataFrame(landsat)
#landsat = landsat + datetime.timedelta(hours=10) #convert to local time (Aus eastern standard time) = utc + 10 hours
#landsat

In [6]:
# Sentinel 2a overpass prediction for 32 * the overpass frequency, this is to give a similar total time to the L8 prediction

Sentinel_2A = []  
for i in range(32):
    Sentinel_2A.append(sentinel2a_startdate + s2a_timestep * (i))
    
Sentinel_2A = pd.DataFrame(Sentinel_2A)
#Sentinel_2A = Sentinel_2A + datetime.timedelta(hours=10) #convert to local time (Aus eastern standard time) = utc + 10 hours
#Sentinel_2A  ### to AEDT, add 11h not 10 ###

In [7]:
# Sentinel 2b

Sentinel_2B = []
for i in range(32):
    Sentinel_2B.append(sentinel2b_startdate + s2b_timestep * (i))
    
Sentinel_2B = pd.DataFrame(Sentinel_2B)
#Sentinel_2B = Sentinel_2B + datetime.timedelta(hours=10) #convert to local time (Aus eastern standard time) = utc + 10 hours
#Sentinel_2B  ### to AEDT, add 11h not 10 ###

In [8]:
# Landasat_2
# Prediction for L8 overpasses at sites which are covered by more than 1 overpass in a 16-day period.
# These sites are Dharawal, Dookie, Fowlers_Gap, Mullion & Winton 

landsat_2 = []
for i in range(20):
    landsat_2.append(l8_startdate_2 + l8_timestep*(i))

landsat_2 = pd.DataFrame(landsat_2)
#landsat_2 = landsat_2 + datetime.timedelta(hours=10)
#landsat_2  ### to AEDT, add 11h not 10 ###

In [9]:
# Sentinel_2A_2
# Sentinel 2a secondary (Dookie, Fowlers_Gap, Lake_Lefroy, Mullion, Tumbarumba)

Sentinel_2A_2 = []  
for i in range(32):
    Sentinel_2A_2.append(sentinel2a_2_startdate + s2a_timestep * (i))
    
Sentinel_2A_2 = pd.DataFrame(Sentinel_2A_2)
#Sentinel_2A_2 = Sentinel_2A_2 + datetime.timedelta(hours=10)
#Sentinel_2A_2  ### to AEDT, add 11h not 10 ###

In [10]:
# Sentinel_2B_2
# Sentinel 2b secondary (Blanchetown, Lake_George, Narrabundah, Pinnacles)

Sentinel_2B_2 = []
for i in range(32):
    Sentinel_2B_2.append(sentinel2b_2_startdate + s2b_timestep * (i))
    
Sentinel_2B_2 = pd.DataFrame(Sentinel_2B_2)
#Sentinel_2B_2 = Sentinel_2B_2 + datetime.timedelta(hours=10)
#Sentinel_2B_2  ### to AEDT, add 11h not 10 ###

### Step 2 - for sites that recieve more than 1 OP in an orbital period, these must be appended to the dataframe.
- You must also specify the field site that this happens first for, for each satellite. ie the field site "Narrabundah" is the first to have an extra L8 OP (actually in the same path as Mullion, -> can use whichever site). This is evident from your input file. See the e.g file "overpass_input.xlsx"

#### Edit the following 3 cells appropriately, altering the .sort_values(by='field_site')

In [11]:
# Combine Landsat 8 data (base plus extra overpasses)

L8_combined2 = landsat.append(landsat_2)
drop_label_L8 = L8_combined2.reset_index(drop=True)
L8_combined2 = drop_label_L8.sort_values(by='Narrabundah') # specify first site which gets extra OP's to sort by
L8_combined2.index.names = ['Landsat_8']
#L8_combined2

In [12]:
# Combine Sentinel 2A data (base plus extra overpasses)

S2A_combined = Sentinel_2A.append(Sentinel_2A_2)
drop_label_S2A = S2A_combined.reset_index(drop=True)
S2A_combined = drop_label_S2A.sort_values(by='Mullion') # specify first site which gets extra OP's to sort by
S2A_combined.index.names = ['Sentinel_2A']
#S2A_combined

In [13]:
# Combine Sentinel 2B data (base plus extra overpasses)

S2B_combined = Sentinel_2B.append(Sentinel_2B_2)
drop_label_S2B = S2B_combined.reset_index(drop=True)
S2B_combined = drop_label_S2B.sort_values(by='Mullion') # specify first site which gets extra OP's to sort by
S2B_combined.index.names = ['Sentinel_2B']
#S2B_combined

In [14]:
S2A_combined['Sat'] = 'S2A'
S2B_combined['Sat'] = 'S2B'
L8_combined2['Sat'] = 'L8'

In [15]:
combined1 = S2B_combined.append(S2A_combined)
combined = combined1.append(L8_combined2)

In [16]:
df = pd.DataFrame(combined)

In [17]:
# Add time dummy - timestep every 2 days to order data by. This allows the output to be ordered in a sensible, human-readable way
timedummy = []
t0 = datetime.date(2019, 11, 1)
dummystep = datetime.timedelta(days=2)

for i in range(300):
    timedummy.append(t0 + dummystep * (i))
    
timedummy = pd.DataFrame(timedummy)

In [18]:
df['DateStep'] = timedummy
df.head(5)

Site,Blanchetown,Dharawal,Dookie,Fowlers_Gap,Lake_George,Lake_Lefroy,Mitchell_Downs,Mullion,Narrabundah,Pinnacles,Tumbarumba,Winton,Sat,DateStep
0,2019-08-12 00:39:00,2019-05-12 23:59:00,2019-02-12 00:20:00,2019-12-18 00:38:00,2019-12-15 23:58:00,2019-12-22 01:58:00,2019-12-18 00:36:00,2019-12-15 23:58:00,2019-12-15 23:58:00,2019-12-21 02:28:00,2019-12-19 00:09:00,2019-12-18 00:36:00,S2B,2019-11-01
32,2019-12-11 00:49:00,NaT,NaT,NaT,NaT,NaT,NaT,2019-12-19 00:07:00,NaT,2019-12-18 02:19:00,NaT,NaT,S2B,2020-01-04
1,2019-08-22 00:39:00,2019-05-22 23:59:00,2019-02-22 00:20:00,2019-12-28 00:38:00,2019-12-25 23:58:00,2020-01-01 01:58:00,2019-12-28 00:36:00,2019-12-25 23:58:00,2019-12-25 23:58:00,2019-12-31 02:28:00,2019-12-29 00:09:00,2019-12-28 00:36:00,S2B,2019-11-03
33,2019-12-21 00:49:00,NaT,NaT,NaT,NaT,NaT,NaT,2019-12-29 00:07:00,NaT,2019-12-28 02:19:00,NaT,NaT,S2B,2020-01-06
2,2019-09-01 00:39:00,2019-06-01 23:59:00,2019-03-04 00:20:00,2020-01-07 00:38:00,2020-01-04 23:58:00,2020-01-11 01:58:00,2020-01-07 00:36:00,2020-01-04 23:58:00,2020-01-04 23:58:00,2020-01-10 02:28:00,2020-01-08 00:09:00,2020-01-07 00:36:00,S2B,2019-11-05


### Step 3 - select the field sites you want to output. 
For new field sites, comment "#" out the old, add the new in the same format.

In [19]:
# Create a new df for each field site of interest. To add a new field site, add in same format ie 
# "Field_Site = df[['Field_Site', 'Sat', 'DateStep']].copy()"

#Blanchetown = df[['Blanchetown', 'Satellite', 'DateStep']].copy()
#Dharawal = df[['Dharawal', 'Satellite', 'DateStep']].copy()
#Dookie = df[['Dookie', 'Satellite', 'DateStep']].copy()
#Fowlers_Gap = df[['Fowlers_Gap', 'Satellite', 'DateStep']].copy()
Lake_George = df[['Lake_George', 'Sat', 'DateStep']].copy()
#Lake_Lefroy = df[['Lake_Lefroy', 'Satellite', 'DateStep']].copy()
#Mitchell_Downs = df[['Mitchell_Downs', 'Satellite', 'DateStep']].copy()
Mullion = df[['Mullion', 'Sat', 'DateStep']].copy()
Narrabundah = df[['Narrabundah', 'Sat', 'DateStep']].copy()
#Pinnacles = df[['Pinnacles', 'Satellite', 'DateStep']].copy()
Tumbarumba = df[['Tumbarumba', 'Sat', 'DateStep']].copy()
#Winton = df[['Winton', 'Satellite', 'DateStep']].copy()

In [20]:
# Reorder by date for each site, include satellite tags for each date

#Blanchetown = Blanchetown.sort_values(by=['Blanchetown', 'DateStep'])
#Dharawal = Dharawal.sort_values(by=['Dharawal', 'DateStep'])
#Dookie = Dookie.sort_values(by=['Dookie', 'DateStep'])
#Fowlers_Gap = Fowlers_Gap.sort_values(by=['Fowlers_Gap', 'DateStep'])
Lake_George = Lake_George.sort_values(by=['Lake_George', 'DateStep'])
#Lake_Lefroy = Lake_Lefroy.sort_values(by=['Lake_Lefroy', 'DateStep'])
#Mitchell_Downs = Mitchell_Downs.sort_values(by=['Mitchell_Downs', 'DateStep'])
Mullion = Mullion.sort_values(by=['Mullion', 'DateStep'])
Narrabundah = Narrabundah.sort_values(by=['Narrabundah', 'DateStep'])
#Pinnacles = Pinnacles.sort_values(by=['Pinnacles', 'DateStep'])
Tumbarumba = Tumbarumba.sort_values(by=['Tumbarumba', 'DateStep'])
#Winton = Winton.sort_values(by=['Winton', 'DateStep'])

In [21]:
# Check that ordering is sensible

Lake_George.head(5)

Site,Lake_George,Sat,DateStep
0,2019-11-29 23:43:00,L8,2019-11-01
1,2019-12-15 23:43:00,L8,2019-11-03
0,2019-12-15 23:58:00,S2B,2019-11-01
0,2019-12-20 23:58:00,S2A,2019-11-01
1,2019-12-25 23:58:00,S2B,2019-11-03


### Step 4 - output individual field site.
- if you wish to output multiple field sites, skip this step.

In [22]:
# Use this cell to ouput an individual field site

# Write individual site to excel:

#Blanchetown.to_excel('Blanchetown_2020.xlsx')
#Dharawal.to_excel('Dharawal_2020.xlsx')
#Dookie.to_excel('Dookie_2020.xlsx')
#Fowlers_Gap.to_excel('Fowlers_Gap_2020.xlsx')
#Lake_George.to_excel('Lake_George_2020.xlsx')
#Lake_Lefroy.to_excel('Lake_Lefroy_2020.xlsx')
#Mitchell_Downs.to_excel('Mitchell_Downs_2020.xlsx')
#Mullion.to_excel('Mullion_2020.xlsx')
#Narrabundah.to_excel('Narrabundah_2020.xlsx')
#Pinnacles.to_excel('Pinnacles_2020.xlsx')
#Tumbarumba.to_excel('Tumbarumba_2020.xlsx')
#Winton.to_excel('Winton_2020.xlsx')

### Step 5 - order combined dataset

- Make sure to have the site with most rows first! Otherwise the data wil be 'squished' by the shorter column when merging
- This shouold be the site with most overpasses, ie the most rows populated in the input file. For the supplied "overpass_input.xlsx" this is the "Mullion" field site
- Follow this process to append each dataset - in this example we are interested in 4 field sites - Lake George, Mullion, Narrabundah and Tumbarumba. 

In [23]:
# Mullion has the most overpasses so is the longest dataset and goes first. Choose a shorter dataset to merge right on. 
# "Lake_George" has much fewer OP's so use this. 
merged = pd.merge(left=Mullion,right=Lake_George, how='outer',left_on='Mullion', right_on='Lake_George')
merged.head(5)

Site,Mullion,Sat_x,DateStep_x,Lake_George,Sat_y,DateStep_y
0,2019-11-20 23:49:00,L8,2019-12-11,NaT,NaN,NaN
1,2019-11-29 23:43:00,L8,2019-11-01,2019-11-29 23:43:00,L8,2019-11-01
2,2019-12-06 23:49:00,L8,2019-12-13,NaT,NaN,NaN
3,2019-12-14 00:07:00,S2A,2020-01-04,NaT,NaN,NaN
4,2019-12-15 23:43:00,L8,2019-11-03,2019-12-15 23:43:00,L8,2019-11-03


In [24]:
merged_2 = pd.merge(left=merged,right=Narrabundah, how='outer',left_on='Mullion', right_on='Narrabundah')
merged_2.head(5)

Site,Mullion,Sat_x,DateStep_x,Lake_George,Sat_y,DateStep_y,Narrabundah,Sat,DateStep
0,2019-11-20 23:49:00,L8,2019-12-11,NaT,NaN,NaN,2019-11-20 23:49:00,L8,2019-12-11
1,2019-11-29 23:43:00,L8,2019-11-01,2019-11-29 23:43:00,L8,2019-11-01,2019-11-29 23:43:00,L8,2019-11-01
2,2019-12-06 23:49:00,L8,2019-12-13,NaT,NaN,NaN,2019-12-06 23:49:00,L8,2019-12-13
3,2019-12-14 00:07:00,S2A,2020-01-04,NaT,NaN,NaN,NaT,NaN,NaN
4,2019-12-15 23:43:00,L8,2019-11-03,2019-12-15 23:43:00,L8,2019-11-03,2019-12-15 23:43:00,L8,2019-11-03


In [25]:
merged_3 = pd.merge(left=merged_2,right=Tumbarumba, how='outer',left_on='Mullion', right_on='Tumbarumba')
# Need to have left / right keys match to merge
merged_3.head(10)

Site,Mullion,Sat_x,DateStep_x,Lake_George,Sat_y,DateStep_y,Narrabundah,Sat_x,DateStep_x,Tumbarumba,Sat_y,DateStep_y
0,2019-11-20 23:49:00,L8,2019-12-11,NaT,NaN,NaN,2019-11-20 23:49:00,L8,2019-12-11,NaT,NaN,NaN
1,2019-11-29 23:43:00,L8,2019-11-01,2019-11-29 23:43:00,L8,2019-11-01,2019-11-29 23:43:00,L8,2019-11-01,NaT,NaN,NaN
2,2019-12-06 23:49:00,L8,2019-12-13,NaT,NaN,NaN,2019-12-06 23:49:00,L8,2019-12-13,NaT,NaN,NaN
3,2019-12-14 00:07:00,S2A,2020-01-04,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN
4,2019-12-15 23:43:00,L8,2019-11-03,2019-12-15 23:43:00,L8,2019-11-03,2019-12-15 23:43:00,L8,2019-11-03,NaT,NaN,NaN
5,2019-12-15 23:58:00,S2B,2019-11-01,2019-12-15 23:58:00,S2B,2019-11-01,2019-12-15 23:58:00,S2B,2019-11-01,NaT,NaN,NaN
6,2019-12-19 00:07:00,S2B,2020-01-04,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN
7,2019-12-20 23:58:00,S2A,2019-11-01,2019-12-20 23:58:00,S2A,2019-11-01,2019-12-20 23:58:00,S2A,2019-11-01,NaT,NaN,NaN
8,2019-12-22 23:49:00,L8,2019-12-15,NaT,NaN,NaN,2019-12-22 23:49:00,L8,2019-12-15,NaT,NaN,NaN
9,2019-12-24 00:07:00,S2A,2020-01-06,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN


In [26]:
merged_3.drop('DateStep_x', axis=1, inplace=True)
merged_3.drop('DateStep_y', axis=1, inplace=True)

In [27]:
merged_3.head(5)

Site,Mullion,Sat_x,Lake_George,Sat_y,Narrabundah,Sat_x,Tumbarumba,Sat_y
0,2019-11-20 23:49:00,L8,NaT,NaN,2019-11-20 23:49:00,L8,NaT,NaN
1,2019-11-29 23:43:00,L8,2019-11-29 23:43:00,L8,2019-11-29 23:43:00,L8,NaT,NaN
2,2019-12-06 23:49:00,L8,NaT,NaN,2019-12-06 23:49:00,L8,NaT,NaN
3,2019-12-14 00:07:00,S2A,NaT,NaN,NaT,NaN,NaT,NaN
4,2019-12-15 23:43:00,L8,2019-12-15 23:43:00,L8,2019-12-15 23:43:00,L8,NaT,NaN


### Step 6 - specify output file name
- Insert desired file name below
- Check output above that the predictor has worked. Days should match between sites, with gaps inserted as "NaT"
- This allows dual overpasses to be identified, across and within field sites

In [28]:
merged_3.to_csv('combined_UTC_OP.csv')

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** Dec 2020

## Tags

**Tags**: :index:`sentinel 2`, :index:`landsat 8`